In [ ]:
# import the necessary libraries
from torchvision.transforms import RandomHorizontalFlip
from torch.utils.data import WeightedRandomSampler
from sklearn.metrics import classification_report
from torchvision.transforms import RandomCrop
from torchvision.transforms import Grayscale
from torchvision.transforms import ToTensor
from torch.utils.data import random_split
from torch.utils.data import DataLoader
# from neuraspike import config as cfg
# from neuraspike import EarlyStopping
# from neuraspike import LRScheduler
# from spike import EmotionNet
from torchvision import transforms
from torchvision import datasets
import matplotlib.pyplot as plt
from collections import Counter
from datetime import datetime
from torch.optim import SGD, Adam
import torch.nn as nn
import pandas as pd
import argparse
import torch
import math
import warnings

In [37]:
TRAIN_DIRECTORY = "/mnt/e/nima_bsl/face/Latest-Version/Latest-Version/dataset/train"
TEST_DIRECTORY = "/mnt/e/nima_bsl/face/Latest-Version/Latest-Version/dataset/test"
train_transform = transforms.Compose([
    Grayscale(num_output_channels=1),
    RandomHorizontalFlip(),
    RandomCrop((48, 48)),
    ToTensor()
])

test_transform = transforms.Compose([
    Grayscale(num_output_channels=1),
    ToTensor()
])

# load all the images within the specified folder and apply different augmentation
train_data = datasets.ImageFolder(TRAIN_DIRECTORY, transform=train_transform)
test_data = datasets.ImageFolder(TEST_DIRECTORY, transform=test_transform)

# extract the class labels and the total number of classes
classes = train_data.classes
classes

['angry', 'fear', 'happy', 'neutral', 'sad', 'surprise']

In [38]:
TRAIN_SIZE = .8
VAL_SIZE = .2
num_of_classes = len(classes)
print(f"[INFO] Class labels: {classes}")

# use train samples to generate train/validation set
num_train_samples = len(train_data)
train_size = math.floor(num_train_samples * TRAIN_SIZE)
val_size = math.ceil(num_train_samples * VAL_SIZE)
print(f"[INFO] Train samples: {train_size} ...\t Validation samples: {val_size}...")

# randomly split the training dataset into train and validation set
train_data, val_data = random_split(train_data, [train_size, val_size])

# modify the data transform applied towards the validation set
val_data.dataset.transforms = test_transform

[INFO] Class labels: ['angry', 'fear', 'happy', 'neutral', 'sad', 'surprise']
[INFO] Train samples: 22967 ...	 Validation samples: 5742...


In [ ]:
train_classes = [label for _, label in train_data]

# count each labels within each classes
class_count = Counter(train_classes)
print(f"[INFO] Total sample: {class_count}")

In [ ]:
class_weight = torch.Tensor([len(train_classes) / c
                             for c in pd.Series(class_count).sort_index().values])

In [42]:
BATCH_SIZE = 16
train_dataloader = DataLoader(train_data, batch_size=BATCH_SIZE)
val_dataloader = DataLoader(val_data, batch_size=BATCH_SIZE)
test_dataloader = DataLoader(test_data, batch_size=BATCH_SIZE)

In [43]:
model = EmotionNet(num_of_channels=1, num_of_classes=num_of_classes)
model = model.to(device)

NameError: name 'EmotionNet' is not defined